In [1]:
import tensorflow as tf
import tensorflow_probability as tfp

tfd = tfp.distributions
tfpl = tfp.layers

In [2]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Conv2D, MaxPooling2D, Flatten, Dropout
from tensorflow.keras.optimizers import Adam
import os
import numpy as np
import matplotlib.pyplot as plt


In [ ]:
#Ignore next to cells if you have already splitted your dataset, else add it to split

In [3]:
#input_dir = os.path.join('D:/Gugan/kaggle brain MRI dataset/')
#output_dir = os.path.join('D:/Gugan/kaggle brain MRI dataset/splitted')

In [4]:
#splitfolders.ratio(input_dir, output=output_dir, seed=1337, ratio=(.8, .2), group_prefix=None)

In [5]:
train_dir = os.path.join('D:/second paper/kaggle brain MRI dataset/splitted/train/')
test_dir = os.path.join('D:/second paper/kaggle brain MRI dataset/splitted/test/')

In [6]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator

In [7]:
def image_generator(train_parent_directory, test_parent_directory):
    
    train_datagen = ImageDataGenerator(rescale=1/255)
    test_datagen = ImageDataGenerator(rescale=1/255)
    
    train_generator = train_datagen.flow_from_directory(train_parent_directory,
                                  target_size = (300,300),
                                  batch_size = 32,
                                  class_mode = 'categorical',
                                  subset='training')
 
    
    test_generator = test_datagen.flow_from_directory(test_parent_directory,
                                 target_size=(300,300),
                                 batch_size = 32,
                                 class_mode = 'categorical')    
    
    return train_generator, test_generator

In [9]:
train_generator, test_generator = image_generator(train_dir, test_dir)

Found 202 images belonging to 2 classes.
Found 51 images belonging to 2 classes.


In [10]:
print(len(train_generator))
print(len(test_generator))

7
2


### Simple 4 layer CNN

In [18]:
model = Sequential([
    Conv2D(input_shape=(300,300,3), filters=8, kernel_size=16, activation='relu'),
    MaxPooling2D(2,2),
    Conv2D(32, (3,3), activation='relu'),
    MaxPooling2D(2,2),
    Conv2D(64, (3,3), activation='relu'),
    MaxPooling2D(2,2),
    Conv2D(64, (3,3), activation='relu'),
    MaxPooling2D(2,2),
    Flatten(),
    Dense(512, activation='relu'),
    Dropout(0.2),
    Dense(units=2, activation='softmax')
])
model.summary()

callback = tf.keras.callbacks.EarlyStopping(monitor = "val_accuracy", min_delta = 0.01, patience = 15, mode = "max", restore_best_weights = True)

Model: "sequential_2"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_4 (Conv2D)            (None, 285, 285, 8)       6152      
_________________________________________________________________
max_pooling2d_8 (MaxPooling2 (None, 142, 142, 8)       0         
_________________________________________________________________
conv2d_5 (Conv2D)            (None, 140, 140, 32)      2336      
_________________________________________________________________
max_pooling2d_9 (MaxPooling2 (None, 70, 70, 32)        0         
_________________________________________________________________
conv2d_6 (Conv2D)            (None, 68, 68, 64)        18496     
_________________________________________________________________
max_pooling2d_10 (MaxPooling (None, 34, 34, 64)        0         
_________________________________________________________________
conv2d_7 (Conv2D)            (None, 32, 32, 64)       

In [ ]:
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])
history = model.fit(
      train_generator,
      steps_per_epoch=7,
    validation_data = test_generator,
    validation_steps = 2,
      epochs=100,
    callbacks = [callback]
)


Epoch 1/100


In [ ]:
model.evaluate(test_generator)

In [ ]:
!pip install opencv-python

In [ ]:
import cv2

def import_and_predict(image_data, label):
    
    #read image
    img = cv2.imread(image_data)
    img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB) 
    
    #show the image
    plt.imshow(img)
    plt.axis('off')
    
    # resize and reshape the image
    img_resize = (cv2.resize(img, dsize=(300, 300), interpolation=cv2.INTER_CUBIC))/255.
    
    img_reshape = img_resize[np.newaxis,...]
    
    #predict the image
    prediction = model.predict(img_reshape)
    print(prediction)
    
    label_prediction = label[np.argmax(prediction)]
    
    return label_prediction

In [ ]:
label = os.listdir(test_dir)
label

In [ ]:
image1_dir = os.path.join(test_dir+'yes/Y244.jpg')
image1_dir

In [ ]:
image2_dir = os.path.join(test_dir+'no/no 99.jpg')
image2_dir

In [ ]:
prediction1 = import_and_predict(image1_dir, label)

In [ ]:
prediction1

### Bayesian CNN with Reparameterization layers

In [15]:
divergence_fn = lambda q,p,_:tfd.kl_divergence(q,p)/3457

model_bayes = Sequential([
    
    tfpl.Convolution2DReparameterization(input_shape=(300,300,3), filters=8, kernel_size=16, activation='relu',
                                           kernel_prior_fn = tfpl.default_multivariate_normal_fn,
                                           kernel_posterior_fn=tfpl.default_mean_field_normal_fn(is_singular=False),
                                           kernel_divergence_fn = divergence_fn,
                                           bias_prior_fn = tfpl.default_multivariate_normal_fn,
                                           bias_posterior_fn=tfpl.default_mean_field_normal_fn(is_singular=False),
                                           bias_divergence_fn = divergence_fn),
    MaxPooling2D(2,2),
    tfpl.Convolution2DReparameterization(input_shape=(300,300,3), filters=8, kernel_size=32, activation='relu',
                                           kernel_prior_fn = tfpl.default_multivariate_normal_fn,
                                           kernel_posterior_fn=tfpl.default_mean_field_normal_fn(is_singular=False),
                                           kernel_divergence_fn = divergence_fn,
                                           bias_prior_fn = tfpl.default_multivariate_normal_fn,
                                           bias_posterior_fn=tfpl.default_mean_field_normal_fn(is_singular=False),
                                           bias_divergence_fn = divergence_fn),
    MaxPooling2D(2,2),
    Conv2D(64, (3,3), activation='relu'),
    MaxPooling2D(2,2),
    Conv2D(64, (3,3), activation='relu'),
    MaxPooling2D(2,2),
    Flatten(),
    Dense(512, activation='relu'),
    Dropout(0.2),
    tfpl.DenseReparameterization(units=tfpl.OneHotCategorical.params_size(2), activation=None,
                                    kernel_prior_fn = tfpl.default_multivariate_normal_fn,
                                    kernel_posterior_fn=tfpl.default_mean_field_normal_fn(is_singular=False),
                                    kernel_divergence_fn = divergence_fn,
                                    bias_prior_fn = tfpl.default_multivariate_normal_fn,
                                    bias_posterior_fn=tfpl.default_mean_field_normal_fn(is_singular=False),
                                    bias_divergence_fn = divergence_fn
                                ),
    tfpl.OneHotCategorical(5)
    
])
model_bayes.summary()
callback = tf.keras.callbacks.EarlyStopping(monitor = "val_accuracy", min_delta = 0.01, patience = 15, mode = "max", restore_best_weights = True)

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_reparameterization_2  (None, 285, 285, 8)       12304     
_________________________________________________________________
max_pooling2d_4 (MaxPooling2 (None, 142, 142, 8)       0         
_________________________________________________________________
conv2d_reparameterization_3  (None, 111, 111, 8)       131088    
_________________________________________________________________
max_pooling2d_5 (MaxPooling2 (None, 55, 55, 8)         0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 53, 53, 64)        4672      
_________________________________________________________________
max_pooling2d_6 (MaxPooling2 (None, 26, 26, 64)        0         
_________________________________________________________________
conv2d_3 (Conv2D)            (None, 24, 24, 64)       

In [16]:
def negative_log_likelihood(y_true, y_pred):
    return -y_pred.log_prob(y_true)

model_bayes.compile(loss = negative_log_likelihood,
              optimizer = Adam(learning_rate=0.005),
              metrics = ['accuracy'],
              experimental_run_tf_function = False)

In [ ]:
history_bayes = model_bayes.fit(
      train_generator,
      epochs=300,
      steps_per_epoch = 7,
      validation_data = test_generator,
      validation_steps = 2,
      callbacks = [callback])

In [ ]:
model_bayes.evaluate(test_generator)

In [ ]:
def import_and_predict_bayes(image, true_label):

    #read image
    img = cv2.imread(image)
    img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB) 
    
    #show the image
    plt.imshow(img)
    plt.axis('off')
    
    img_resize = (cv2.resize(img, dsize=(300, 300), interpolation=cv2.INTER_CUBIC))/255.
    
    predicted_probabilities = np.empty(shape=(300, 2))
    
    for i in range(300):
        
        predicted_probabilities[i] = model_bayes(img_resize[np.newaxis,...]).mean().numpy()[0]
        
    pct_2p5 = np.array([np.percentile(predicted_probabilities[:, i], 2.5) for i in range(2)])
    pct_97p5 = np.array([np.percentile(predicted_probabilities[:, i], 97.5) for i in range(2)])
    
    fig, ax = plt.subplots(figsize=(12, 6))
    bar = ax.bar(np.arange(2), pct_97p5, color='red')
    bar[true_label].set_color('green')
    bar = ax.bar(np.arange(2), pct_2p5-0.02, color='white', linewidth=1, edgecolor='white')
    ax.set_xticklabels([''] + [x for x in label])
    ax.set_ylim([0, 1])
    ax.set_ylabel('Probability')
    plt.show()

In [ ]:
import_and_predict_bayes(image2_dir, label.index('no'))